In [1]:
import polars as pl

In [ ]:
# Read processed data
df = pl.read_csv("../data/processed/day-01.csv")

In [5]:
# Calculate the distance between the left and right columns
df = df.with_columns((pl.col('left') - pl.col('right')).abs().alias('distance'))

In [8]:
# Calculate the sum of the distances
df.select(pl.col('distance').sum())

distance
i64
2756096
